In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!source /content/drive/MyDrive/colab_env/bin/activate

/bin/bash: /content/drive/MyDrive/colab_env/bin/activate: No such file or directory


In [ ]:
! pip install geemap wxee

In [138]:
import xarray as xr
import ee
import itertools
import pickle
import pandas as pd
import wxee
import rasterio

In [139]:
LAT_MIN = 25
LAT_MAX = 45
LON_MIN = -125
LON_MAX = -65

In [140]:
soil_wc = xr.open_dataset('/content/drive/MyDrive/w/soil_water_content_1979001.nc')
soil_wc

<xarray.Dataset>
Dimensions:                        (x: 600, y: 200, time: 1)
Coordinates:
  * x                              (x) float32 -124.9 -124.8 ... -65.15 -65.05
  * y                              (y) float32 44.95 44.85 44.75 ... 25.15 25.05
  * time                           (time) datetime64[ns] 1979-01-01
    spatial_ref                    int64 ...
Data variables:
    volumetric_soil_water_layer_1  (time, y, x) float64 ...
    volumetric_soil_water_layer_3  (time, y, x) float64 ...
    volumetric_soil_water_layer_4  (time, y, x) float64 ...
Attributes:
    AREA_OR_POINT:           Area
    TIFFTAG_RESOLUTIONUNIT:  1 (unitless)
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    _FillValue:              -32768.0
    scale_factor:            1.0
    add_offset:              0.0

In [141]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file="/content/drive/MyDrive/w/private-key.json"
)

ee.Initialize(credentials)


In [142]:
# Soil depths [in cm] where we have data.
olm_depths = [0, 10, 30, 60, 100, 200]

# Names of bands associated with reference depths.
olm_bands = ["b" + str(sd) for sd in olm_depths]

def get_soil_prop(param):
    """
    This function returns soil properties image
    param (str): must be one of:
        "sand"     - Sand fraction
        "clay"     - Clay fraction
        "orgc"     - Organic Carbon fraction
    """
    if param == "sand":  # Sand fraction [%w]
        snippet = "OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02"
        # Define the scale factor in accordance with the dataset description.
        scale_factor = 1 * 0.01

    elif param == "clay":  # Clay fraction [%w]
        snippet = "OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02"
        # Define the scale factor in accordance with the dataset description.
        scale_factor = 1 * 0.01

    elif param == "orgc":  # Organic Carbon fraction [g/kg]
        snippet = "OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02"
        # Define the scale factor in accordance with the dataset description.
        scale_factor = 5 * 0.001  # to get kg/kg
    else:
        return print("error")

    # Apply the scale factor to the ee.Image.
    dataset = ee.Image(snippet).multiply(scale_factor)

    return dataset


# Image associated with the sand content.
sand = get_soil_prop("sand")

# Image associated with the clay content.
clay = get_soil_prop("clay")

# Image associated with the organic carbon content.
orgc = get_soil_prop("orgc")

In [143]:
lat_of_points = list(soil_wc.y.values.flatten())
lon_of_points = list(soil_wc.x.values.flatten())

grid = [(float(i), float(j)) for i, j in itertools.product(lon_of_points, lat_of_points)]
x_values = [t[0] for t in grid]
y_values = [t[1] for t in grid]


In [144]:
shape = ee.Geometry.Rectangle([LON_MAX,LAT_MIN,LON_MIN,LAT_MAX])
s = sand.clip(shape).set("system:time_start", ee.Date("1979-01-01")).wx.to_xarray(scale=10000, crs=f'EPSG:{crs}')
xds_repr_match_sand = s.rio.reproject_match(soil_wc, resampling = rasterio.enums.Resampling.nearest)

Downloading:   0%|          | 0.00/680k [00:00<?, ?iB/s]

In [145]:
c = clay.clip(shape).set("system:time_start", ee.Date("1979-01-01")).wx.to_xarray(scale=10000, crs=f'EPSG:{crs}')
xds_repr_match_clay = c.rio.reproject_match(soil_wc, resampling = rasterio.enums.Resampling.nearest)

Downloading:   0%|          | 0.00/547k [00:00<?, ?iB/s]

In [146]:
orgm = orgc.multiply(1.724)
o = orgm.clip(shape).set("system:time_start", ee.Date("1979-01-01")).wx.to_xarray(scale=10000, crs=f'EPSG:{crs}')
xds_repr_match_orgm = o.rio.reproject_match(soil_wc, resampling = rasterio.enums.Resampling.nearest)


Downloading:   0%|          | 0.00/301k [00:00<?, ?iB/s]

In [147]:
theta1500_list = []
theta33_list = []
wp_list = []
fc_list = []

for key in olm_bands:
    S = xds_repr_match_sand[key]
    C = xds_repr_match_clay[key]
    OM = xds_repr_match_orgm[key]

    theta_1500ti = -0.024 * S + 0.487 * C + 0.006 * OM + 0.005 * (S * OM) - 0.013 * (C * OM) + 0.068 * (S * C) + 0.031
    theta1500_list.append(theta_1500ti)

    wpi = (theta_1500ti + ( 0.14 * theta_1500ti - 0.002)) * 100
    wp_list.append(wpi)

    theta_33ti = -0.251 * S + 0.195 * C + 0.011 * OM + 0.006 * (S * OM) - 0.027 * (C * OM)+ 0.452 * (S * C) + 0.299
    theta33_list.append(theta_33ti)

    fci = (theta_33ti + (1.283 * theta_33ti * theta_33ti - 0.374 * theta_33ti - 0.015)) * 100
    fc_list.append(fci)


combined_dataset_fc = xr.merge(fc_list)
combined_dataset_wp = xr.merge(wp_list)

In [148]:
combined_dataset_fc.to_netcdf("/content/drive/MyDrive/w/field_capacity_1979001.nc")
combined_dataset_wp.to_netcdf("/content/drive/MyDrive/w/wilting_point_1979001.nc")